In [1]:
import pandas as pd
from datetime import datetime, timedelta, timezone
import os
from gql import gql
import re

In [2]:
from gql import Client
from gql.transport.aiohttp import AIOHTTPTransport
from gql.transport.httpx import HTTPXTransport


class GraphQLClient:
    def __init__(self, async_mode: bool = True):
        GRAPHQL_ENDPOINT = os.getenv(
            "GRAPHQL_ENDPOINT", "http://localhost:8080/v1/graphql"
        )
        HASURA_GRAPHQL_ADMIN_SECRET = os.getenv(
            "HASURA_GRAPHQL_ADMIN_SECRET", "superdupersecuresecret"
        )
        params = {
            "url": GRAPHQL_ENDPOINT,
            "headers": {"x-hasura-admin-secret": HASURA_GRAPHQL_ADMIN_SECRET},
        }

        if async_mode:
            transport = AIOHTTPTransport(**params)  # type: ignore
        else:
            transport = HTTPXTransport(**params)  # type: ignore

        self.client = Client(transport=transport, fetch_schema_from_transport=True)

    def getClient(self):
        return self.client


In [3]:
# Import excel file data.xlsx
products = pd.read_excel("data.xlsx", sheet_name=6)

In [4]:
products.columns

Index(['Produkt ID', 'Name', 'Eigene Produktnummer', 'Einheit'], dtype='object')

In [5]:
products.rename(columns={
    "Produkt ID": "productId",
    "Name": "name",
    "Einheit": "unit",
}, inplace=True)
products["externalArticleNumber"] = products["productId"].astype(str)
products["productId"] = products["productId"].astype(str)
products["name"] = products["name"].str.strip()
products["unit"] = products["unit"].str.strip()
products["createdAt"] = datetime.now(timezone.utc).isoformat()
products["updatedAt"] = datetime.now(timezone.utc).isoformat()
products["userId"] = "cm6p6u49o0000k9wtbwgfft3h"
products["retired"] = False
products["name"] = products["name"].str.replace(r"\s{2,}", " ", regex=True)
products.drop(columns=["Eigene Produktnummer"], inplace=True)

In [6]:
product_insert_query = gql("""
mutation MyMutation($objects: [masterdata_Product_insert_input!] = {}) {
  insert_masterdata_Product(objects: $objects, on_conflict: {constraint: Product_pkey, update_columns: name}) {
    affected_rows
  }
}
""")

client = GraphQLClient(async_mode=False).getClient()
with client as session:
    result = session.execute(
        product_insert_query,
        variable_values={"objects": products.to_dict(orient="records")},
    )
    print(result)
    print(f"Inserted {result['insert_masterdata_Product']['affected_rows']} products.")

{'insert_masterdata_Product': {'affected_rows': 2073}}
Inserted 2073 products.


In [7]:
locations_df = pd.read_excel("data.xlsx", sheet_name=5)
locations_df.rename(columns={
    "Ort ID": "locationId",
    "Name": "name",
}, inplace=True)
locations_df["locationId"] = locations_df["locationId"].astype(str)
locations_df.drop(columns=["name"], inplace=True, errors='ignore')
locations_df["createdAt"] = datetime.now(timezone.utc).isoformat()
locations_df["updatedAt"] = datetime.now(timezone.utc).isoformat() 
locations_df["userId"] = "cm6p6u49o0000k9wtbwgfft3h"

In [8]:
locations_query = gql("""
mutation MyMutation($objects: [masterdata_Location_insert_input!] = {}) {
  insert_masterdata_Location(objects: $objects, on_conflict: {constraint: Location_pkey, update_columns: userId}) {
    affected_rows
    }
}
""")

client = GraphQLClient(async_mode=False).getClient()
with client as session:
    result = session.execute(
        locations_query,
        variable_values={"objects": locations_df.to_dict(orient="records")},
    )
    print(result)
    print(f"Inserted {result['insert_masterdata_Location']['affected_rows']} locations.")


{'insert_masterdata_Location': {'affected_rows': 2}}
Inserted 2 locations.


In [9]:
# Insert assetType into database
with client as session:
    result = session.execute(
        gql("""mutation MyMutation {
            insert_masterdata_AssetType_one(object: {assetTypeId: "machine", name: "machine", isWorker: false, isStorage: false, isMultiProductStorage: false, fixedPosition: true, createdAt: "2025-08-15T14:45:00", userId: "cm6p6u49o0000k9wtbwgfft3h", sizeUnit: "kg", retired: false, updatedAt: "2025-08-15T14:45:00"}) {
              assetTypeId
            }
          }
          """))
    print(result)
    print(f"Inserted asset type: {result['insert_masterdata_AssetType_one']['assetTypeId']}.")

{'insert_masterdata_AssetType_one': {'assetTypeId': 'machine'}}
Inserted asset type: machine.


In [10]:
assets_df = pd.read_excel("data.xlsx", sheet_name=4)

In [11]:
assets_df.rename(columns={
    "Anlagen ID": "assetId",
    "Name": "name",
    "Ort": "locationId"
}, inplace=True)

assets_df["assetTypeId"] = "machine"
assets_df["assetId"] = assets_df["assetId"].astype(str)
assets_df["locationId"] = assets_df["locationId"].astype(str)
assets_df["name"] = assets_df["name"].str.strip()
assets_df["createdAt"] = datetime.now(timezone.utc).isoformat()
assets_df["updatedAt"] = datetime.now(timezone.utc).isoformat()
assets_df["userId"] = "cm6p6u49o0000k9wtbwgfft3h"
assets_df["retired"] = False
assets_df["name"] = assets_df["name"].str.replace(r"\s{2,} ", " ", regex=True)
assets_df.drop(columns=["Anlagentyp"], inplace=True, errors='ignore')

In [12]:
with client as session:
    result = session.execute(
        gql("""
        mutation MyMutation($objects: [masterdata_Asset_insert_input!] = {}) {
            insert_masterdata_Asset(objects: $objects, on_conflict: {constraint: Asset_pkey, update_columns: name}) {
                affected_rows
            }
        }
        """),
        variable_values={"objects": assets_df.to_dict(orient="records")},
    )
    print(result)
    print(f"Inserted {result['insert_masterdata_Asset']['affected_rows']} assets.")

{'insert_masterdata_Asset': {'affected_rows': 11}}
Inserted 11 assets.


In [13]:
production_orders_df = pd.read_excel("data.xlsx", sheet_name=1)
production_orders_df.rename(columns={
    "Auftragsnummer": "productionOrderId",
    "Produkt ID": "mainProductId",
    "Batchnummer": "number",
    "Menge": "actualQuantity",
    "Start": "actualStart",
    "Ende": "actualEnd",
}, inplace=True)


In [14]:
# Convert actualStart and actualEnd from YYYY-MM-DD to datetime object
def convert_to_datetime(date_str):
    if pd.isna(date_str):
        return None
    return datetime.strptime(date_str, "%Y-%m-%d").replace(tzinfo=timezone.utc)
production_orders_df["actualStart"] = production_orders_df["actualStart"].apply(convert_to_datetime)
production_orders_df["actualEnd"] = production_orders_df["actualEnd"].apply(convert_to_datetime)

# Convert datetime objects to ISO format
production_orders_df["actualStart"] = production_orders_df["actualStart"].apply(lambda x: x.isoformat() if x else None)
production_orders_df["actualEnd"] = production_orders_df["actualEnd"].apply(lambda x: x.isoformat() if x else None)
production_orders_df["productionOrderId"] = production_orders_df["productionOrderId"].astype(str)
production_orders_df["mainProductId"] = production_orders_df["mainProductId"].astype(str)
production_orders_df["number"] = production_orders_df["number"].astype(str)
production_orders_df["createdAt"] = datetime.now(timezone.utc).isoformat()
production_orders_df["updatedAt"] = datetime.now(timezone.utc).isoformat()

# delete rows with NaN in mainProductId
production_orders_df.dropna(subset=["mainProductId"], inplace=True)

# set entries with NaN in actualQuantity to 0
production_orders_df["actualQuantity"] = production_orders_df["actualQuantity"].fillna(0)

# only keep touched columns
production_orders_df = production_orders_df[[
    "productionOrderId",
    "mainProductId",
    "number",
    "actualQuantity",
    "actualStart",
    "actualEnd",
    "createdAt",
    "updatedAt"
]]



In [15]:
production_order_query = gql("""
                             mutation MyMutation($objects: [soe_state_ProductionOrder_insert_input!] = {}) {
  insert_soe_state_ProductionOrder(objects: $objects) {
    affected_rows
  }
}
""")
client = GraphQLClient(async_mode=False).getClient()

with client as session:
    result = session.execute(
        production_order_query,
        variable_values={"objects": production_orders_df.to_dict(orient="records")},
    )
    print(result)
    print(f"Inserted {result['insert_soe_state_ProductionOrder']['affected_rows']} production orders.")

{'insert_soe_state_ProductionOrder': {'affected_rows': 15550}}
Inserted 15550 production orders.


In [16]:
tasks_df = pd.read_excel("data.xlsx", sheet_name=0)
tasks_df.rename(columns={
    "Anlage": "assetId",
    "Auftrag": "processRecordId",
    "Start": "actualStart",
    "Ende": "actualEnd",
    "Zustand": "name"
}, inplace=True)


In [17]:
# Ensure both keys are of the same type (string)
tasks_df["processRecordId"] = tasks_df["processRecordId"].astype(str)
production_orders_df["productionOrderId"] = production_orders_df["productionOrderId"].astype(str)

# Join the tasks_df on the production_orders_df by processRecordId = productionOrderId
merged_production_tasks = tasks_df.merge(
    production_orders_df[["productionOrderId", "mainProductId"]],
    left_on="processRecordId",
    right_on="productionOrderId",
    how="left"
)
# count how many production_orders have no tasks
no_tasks_count = production_orders_df[~production_orders_df["productionOrderId"].isin(tasks_df["processRecordId"])]["productionOrderId"].count()
print(f"Number of production orders with no tasks: {no_tasks_count} which is {no_tasks_count / production_orders_df.shape[0] * 100:.2f}% of all production orders.")

Number of production orders with no tasks: 7468 which is 48.03% of all production orders.


In [18]:
# drop all rows where productionOrderId is NaN
merged_production_tasks.dropna(subset=["productionOrderId"], inplace=True)

# only keep rows where name = Rüsten
merged_production_tasks = merged_production_tasks[merged_production_tasks["name"] == "Rüsten"]

In [19]:
# Count how many tasks have no production order after joining them
no_production_order_tasks_count = merged_production_tasks[merged_production_tasks["productionOrderId"].isna()]["processRecordId"].count()
print(f"Number of tasks with no production order: {no_production_order_tasks_count} which is {no_production_order_tasks_count / tasks_df.shape[0] * 100:.2f}% of all tasks.")

Number of tasks with no production order: 0 which is 0.00% of all tasks.


In [ ]:
from uuid import uuid4

inventory_df = pd.read_excel("data.xlsx", sheet_name=3)
inventory_df.rename(
    columns={
        "Produkt ID": "productId",
        "Menge": "amount",
        "Ort": "locationId",
        "Batchnummer": "productionOrderNumber",
        # "Auftragsnummer": "productionOrderId",
        "Letzte Aktualisierung": "updatedAt",
    },
    inplace=True,
)
inventory_df["productId"] = inventory_df["productId"].astype(str)
# drop rows with na in productId or amount
inventory_df.dropna(subset=["productId", "amount"], inplace=True)
inventory_df["locationId"] = inventory_df["locationId"].astype(str)
inventory_df["updatedAt"] = inventory_df["updatedAt"].apply(convert_to_datetime)
inventory_df["updatedAt"] = inventory_df["updatedAt"].apply(
    lambda x: x.isoformat() if x else None
)
# inventory_df["updatedAt"] = datetime.now(timezone.utc).isoformat()
inventory_df["updatedBy"] = "cm6p6u49o0000k9wtbwgfft3h"
inventory_df["inventoryId"] = inventory_df.apply(
    lambda row: str(uuid4()), axis=1
)  # Generate a unique inventoryId for each row

# set unit statically to pcs
inventory_df["unit"] = "pcs"

# drop Auftragsnummer column
inventory_df.drop(columns=["Auftragsnummer"], inplace=True, errors='ignore')

# convert to list of dict
inventory_objects = inventory_df.to_dict(orient="records")

# run through the inventory objects and delete all attributes from each dict if it is none
for obj in inventory_objects:
    keys_to_delete = [
        key
        for key, value in obj.items()
        if value is None
        or (isinstance(value, float) and pd.isna(value))
        or (isinstance(value, str) and value.strip() == "")
        or (isinstance(value, str) and value == 'NaT')  # Also remove zero values
    ]
    for key in keys_to_delete:
        del obj[key]

# count all nan values in the amount column
nan_amount_count = inventory_df["amount"].isna().sum()
print(
    f"Number of NaN values in amount column: {nan_amount_count} which is {nan_amount_count / inventory_df.shape[0] * 100:.2f}% of all inventory records."
)

inventory_query = gql("""
mutation MyMutation($objects: [soe_state_inventory_insert_input!] = {}) {
  insert_soe_state_inventory(objects: $objects, on_conflict: {constraint: inventory_pkey, update_columns: amount}) {
    affected_rows
  }
}
""")

# Execute the inventory query
with client as session:
    result = session.execute(
        inventory_query,
        variable_values={"objects": inventory_objects},
    )
    print(result)
    print(
        f"Inserted {result['insert_soe_state_inventory']['affected_rows']} inventory records."
    )

Number of NaN values in amount column: 0 which is 0.00% of all inventory records.
{'insert_soe_state_inventory': {'affected_rows': 21668}}


KeyError: 'insert_soe_state_Inventory'

In [37]:
demand_df = pd.read_excel("data.xlsx", sheet_name=2)
demand_df.rename(columns={
    "Produkt ID": "productId",
    "Menge": "amount",
    "Benötigte Fertigstellung Produktion": "confirmedDueDate",
    "Auftragsnummer": "orderId",
}, inplace=True)

# statically set the unit to pcs
demand_df["unit"] = "pcs"
demand_df["productId"] = demand_df["productId"].astype(str)
demand_df["orderId"] = demand_df["orderId"].astype(str)
# convert confirmedDueDate to pd.Timestamp
demand_df["confirmedDueDate"] = pd.to_datetime(demand_df["confirmedDueDate"], errors='coerce')
# dump to ISO format
demand_df["confirmedDueDate"] = demand_df["confirmedDueDate"].apply(
    lambda x: x.isoformat() if pd.notna(x) else None
)
demand_df["createdAt"] = datetime.now(timezone.utc).isoformat()
demand_df["updatedAt"] = datetime.now(timezone.utc).isoformat()
demand_df["createdBy"] = "cm6p6u49o0000k9wtbwgfft3h"
demand_df["demandId"] = demand_df.apply(
    lambda row: str(uuid4()), axis=1
)  # Generate a unique demandId for each row
# drop column Produktionsauftragsnummer
demand_df.drop(columns=["Produktionsauftragsnummer"], inplace=True, errors='ignore')
# drop column MTO/MTS
demand_df.drop(columns=["MTO/MTS"], inplace=True, errors='ignore')
# statically set the locationId to ESS
demand_df["locationId"] = "ESS"

# convert to list of dict
demand_objects = demand_df.to_dict(orient="records")

demand_query = gql("""
mutation MyMutation($objects: [soe_state_demand_insert_input!] = {}) {
  insert_soe_state_demand(objects: $objects, on_conflict: {constraint: demand_pkey, update_columns: amount}) {
    affected_rows
  }
}
""")

# Execute the demand query
with client as session:
    result = session.execute(
        demand_query,
        variable_values={"objects": demand_objects},
    )
    print(result)
    print(f"Inserted {result['insert_soe_state_demand']['affected_rows']} demand records.")

{'insert_soe_state_demand': {'affected_rows': 1161}}
Inserted 1161 demand records.
